In [1]:
import re
import torch
from PIL import Image
import gc
import requests
import copy
from PIL import Image
from io import BytesIO
from torch.nn.functional import mse_loss
import numpy as np
import einops
from llava.model.builder import load_pretrained_model
from llava.mm_utils import get_model_name_from_path
from llava.eval.run_llava import eval_model
from llava.conversation import conv_templates
from llava.constants import (
    IMAGE_TOKEN_INDEX,
    DEFAULT_IMAGE_TOKEN,
    DEFAULT_IM_START_TOKEN,
    DEFAULT_IM_END_TOKEN,
    IMAGE_PLACEHOLDER,
)
from llava.mm_utils import (
    process_images,
    tokenizer_image_token,
    get_model_name_from_path,
)

from utils.models.factory import create_model_and_transforms, get_tokenizer
from utils.models.prs_hook import hook_prs_logger


/cluster/apps/vogtlab/users/vstrozzi/software/anaconda/envs/MT/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Could not find the bitsandbytes CUDA binary at PosixPath('/cluster/apps/vogtlab/users/vstrozzi/software/anaconda/envs/MT/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda121.so')
The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.


[2025-02-24 16:25:05,399] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
# Helper functions
def image_parser(image_file, sep=","):
    out = image_file.split(sep)
    return out


def load_image(image_file):
    if image_file.startswith("http") or image_file.startswith("https"):
        response = requests.get(image_file)
        image = Image.open(BytesIO(response.content)).convert("RGB")
    else:
        image = Image.open(image_file).convert("RGB")
    return image


def load_images(image_files):
    out = []
    for image_file in image_files:
        image = load_image(image_file)
        out.append(image)
    return out

device = "cuda"
model_name = "liuhaotian/llava-v1.5-7b"
model_path = "/cluster/work/vogtlab/Group/vstrozzi/cache/models--liuhaotian--llava-v1.5-7b/snapshots/4481d270cc22fd5c4d1bb5df129622006ccd9234/"

## Get LLava Model

In [3]:
### IMPORT: On Biomedcluster change .config under model_path to point towards correct vision_tower clip path
tokenizer, model, image_processor, context_len = load_pretrained_model(
    model_path=model_path,
    model_base=None,
    model_name=get_model_name_from_path(model_name),
    device_map="cpu",  # HERE
)

vision_enc = model.get_vision_tower()

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in vision_enc.parameters()]):,}")


model1 = vision_enc
print(model1.device)

You are using a model of type llava to instantiate a model of type llava_llama. This is not supported for all configurations of models and can yield errors.


LlavaConfig {
  "_name_or_path": "/cluster/work/vogtlab/Group/vstrozzi/cache/models--liuhaotian--llava-v1.5-7b/snapshots/4481d270cc22fd5c4d1bb5df129622006ccd9234/",
  "architectures": [
    "LlavaLlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "freeze_mm_mlp_adapter": false,
  "freeze_mm_vision_resampler": false,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "image_aspect_ratio": "pad",
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_length": 4096,
  "max_position_embeddings": 4096,
  "mm_hidden_size": 1024,
  "mm_projector_type": "mlp2x_gelu",
  "mm_resampler_type": null,
  "mm_use_im_patch_token": false,
  "mm_use_im_start_end": false,
  "mm_vision_select_feature": "patch",
  "mm_vision_select_layer": -2,
  "mm_vision_tower": "/cluster/work/vogtlab/Group/vstrozzi/cache/models--openai--clip-vit-large-patch14-336/snapshots/ce19dc912ca5cd21c8a653c79e251e808ccabcd1/",
  "model_type": "llava_lla

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]/cluster/apps/vogtlab/users/vstrozzi/software/anaconda/envs/MT/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.90it/s]


Model parameters: 303,507,456
cpu


In [4]:
# CLIP model
model_CLIP_name = 'ViT-L-14-336' 
pretrained = "hf"
precision = "fp16"

torch.cuda.empty_cache()
model_CLIP, _, preprocess_clip = create_model_and_transforms(model_CLIP_name, pretrained=pretrained, precision=precision, cache_dir="../cache")

model_CLIP.eval()
context_length = model_CLIP.context_length
# Not needed anymore
vocab_size = model_CLIP.vocab_size
# tokenizer_CLIP = get_tokenizer(model_CLIP_name)

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model_CLIP.visual.parameters()]):,}")
print("Context length:", context_length)
print("Vocab size:", vocab_size)
print("Len of res:", len(model_CLIP.visual.transformer.resblocks))
# Replace with the one we want to use
model2 = model_CLIP.visual
prs = hook_prs_logger(model_CLIP, device, spatial=True, vision_projection=False) # This attach hook to get the residual stream
print(model_CLIP.visual)

Using local files
Model parameters: 304,293,888
Context length: 77
Vocab size: 49408
Len of res: 24
VisionTransformer(
  (patchnorm_pre_ln): Identity()
  (conv1): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
  (patch_dropout): Identity()
  (ln_pre): LayerNorm()
  (transformer): Transformer(
    (resblocks): ModuleList(
      (0-23): 24 x ResidualAttentionBlock(
        (ln_1): LayerNorm()
        (attn): MultiheadAttention(
          (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (ls_1): Identity()
        (ln_2): LayerNorm()
        (mlp): MLP(
          (c_fc): Linear(in_features=1024, out_features=4096, bias=True)
          (gelu): QuickGELUActivation()
          (c_proj): Linear(in_features=4096, out_features=1024, bias=True)
        )
        (ls_2): Identity()
      )
    )
  )
  (ln_post): LayerNorm()
)


In [5]:
import torch
import torch.nn as nn

# Assume model1 and model2 are defined, loaded, and moved to the proper device.
# For example:
#   model1 = ...  # a CLIP encoder-based model with: vision_tower.vision_model.encoder.layers (CLIPEncoderLayer)
#   model2 = ...  # a VisionTransformer model with: transformer.resblocks (ResidualAttentionBlock)

# Set both models to evaluation mode.
model1.eval()
model2.eval()

# ----------------------------------------------------------------------------
# Dictionaries to store activations (outputs) from each model.
activations_model1 = {}
activations_model2 = {}

# ----------------------------------------------------------------------------
# Hook function to capture module outputs.
def get_activation(name, activation_dict):
    def hook(module, input, output):
        # If the module returns a tuple, take the first element.
        if isinstance(output, tuple):
            output = output[0]
        activation_dict[name] = output.detach().cpu()
    return hook

# ----------------------------------------------------------------------------
# Register high-level hooks for Model1 (CLIPEncoder-based)
model1_patch_embed = model1.vision_tower.vision_model.embeddings.patch_embedding
model1_patch_embed.register_forward_hook(get_activation("patch_embedding", activations_model1))

model1_pre_ln = model1.vision_tower.vision_model.pre_layrnorm
model1_pre_ln.register_forward_hook(get_activation("pre_layrnorm", activations_model1))

encoder_layers_model1 = model1.vision_tower.vision_model.encoder.layers
for i, layer in enumerate(encoder_layers_model1):
    layer.register_forward_hook(get_activation(f"encoder_layer_{i}", activations_model1))
    
model1_post_ln = model1.vision_tower.vision_model.post_layernorm
model1_post_ln.register_forward_hook(get_activation("postlayernorm", activations_model1))

# ----------------------------------------------------------------------------
# Register high-level hooks for Model2 (ResidualAttentionBlock-based)
model2_patch_embed = model2.conv1
model2_patch_embed.register_forward_hook(get_activation("conv1", activations_model2))

model2_pre_ln = model2.ln_pre
model2_pre_ln.register_forward_hook(get_activation("ln_pre", activations_model2))

resblocks_model2 = model2.transformer.resblocks
for i, block in enumerate(resblocks_model2):
    block.register_forward_hook(get_activation(f"resblock_{i}", activations_model2))
    
model2_post_ln = model2.ln_post
model2_post_ln.register_forward_hook(get_activation("ln_post", activations_model2))

# ----------------------------------------------------------------------------
# Register internal hooks for each transformer/encoder block.
# For Model2 (ResidualAttentionBlock): capture internal submodule outputs.
for i, block in enumerate(resblocks_model2):
    # First layer normalization.
    block.ln_1.register_forward_hook(get_activation(f"resblock_{i}_ln_1", activations_model2))
    
    # For the attention output, capture the output of the attention module.
    block.attn.register_forward_hook(get_activation(f"resblock_{i}_attn_out", activations_model2))

    # Second layer normalization.
    block.ln_2.register_forward_hook(get_activation(f"resblock_{i}_ln_2", activations_model2))
    
    # MLP internal components.
    block.mlp.c_fc.register_forward_hook(get_activation(f"resblock_{i}_mlp_fc1", activations_model2))
    block.mlp.gelu.register_forward_hook(get_activation(f"resblock_{i}_mlp_gelu", activations_model2))
    block.mlp.c_proj.register_forward_hook(get_activation(f"resblock_{i}_mlp_fc2", activations_model2))

# For Model1 (CLIPEncoderLayer): capture internal submodule outputs.
for i, layer in enumerate(encoder_layers_model1):
    layer.layer_norm1.register_forward_hook(get_activation(f"encoder_layer_{i}_ln1", activations_model1))
    layer.self_attn.out_proj.register_forward_hook(get_activation(f"encoder_layer_{i}_attn_out", activations_model1))
    layer.layer_norm2.register_forward_hook(get_activation(f"encoder_layer_{i}_ln2", activations_model1))
    layer.mlp.fc1.register_forward_hook(get_activation(f"encoder_layer_{i}_mlp_fc1", activations_model1))
    layer.mlp.activation_fn.register_forward_hook(get_activation(f"encoder_layer_{i}_mlp_act", activations_model1))
    layer.mlp.fc2.register_forward_hook(get_activation(f"encoder_layer_{i}_mlp_fc2", activations_model1))

# ----------------------------------------------------------------------------
# Prepare an input for the forward pass.
# Here we create a dummy input; replace with your actual image processing pipeline.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x = 100*torch.randn(1, 3, 336, 336, device=device, dtype=torch.float16)

# ----------------------------------------------------------------------------
# Run a forward pass for both models.
model1.to("cuda")
model1.to
output_model1 = model1(x)
model1.to("cpu")
model2.to("cuda")
output_model2 = model2(x)
model2.to("cpu")

# ----------------------------------------------------------------------------
# Compare high-level activations.
layer_mapping = {
    "patch_embedding": "conv1",    # Model1 patch embedding vs. Model2 conv1
    "pre_layrnorm": "ln_pre",
    "postlayernorm": "ln_post"
}

print(model1)
print(model2)
print("=== High-Level Activations Comparison ===")
for name1, name2 in layer_mapping.items():
    act1 = activations_model1.get(name1)
    act2 = activations_model2.get(name2)
    if act1 is not None and act2 is not None:
        diff = torch.norm(act1 - act2)
        print(f"Model1 {name1}: shape {act1.shape}, norm {torch.norm(act1)}")
        print(f"Model2 {name2}: shape {act2.shape}, norm {torch.norm(act2)}")
        print(f"Difference: {diff.item()}\n")
    else:
        print(f"Missing activation for {name1} or {name2}\n")

# ----------------------------------------------------------------------------
# Compare internal submodule outputs for each block.
print("=== Internal Blocks Comparison ===")
for i in range(24):
    print(f"--- Block {i} ---")
    
    # Compare first layer normalization (ln_1 vs. layer_norm1)
    act1 = activations_model1.get(f"encoder_layer_{i}_ln1")
    act2 = activations_model2.get(f"resblock_{i}_ln_1")
    if act1 is not None and act2 is not None:
        diff = torch.norm(act1 - act2.to(dtype=torch.float32))
        print(f"LayerNorm1 difference: {diff.item()}")
    else:
        print("Missing ln1 activation")
    
    # Compare attention's output projection.
    act1 = activations_model1.get(f"encoder_layer_{i}_attn_out")
    act2 = activations_model2.get(f"resblock_{i}_attn_out")
    if act1 is not None and act2 is not None:
        diff = torch.norm(act1 - act2.to(dtype=torch.float32))
        print(f"Attention Out Projection difference: {diff.item()}")
    else:
        print("Missing attention out activation")
    
    # Compare second layer normalization.
    act1 = activations_model1.get(f"encoder_layer_{i}_ln2")
    act2 = activations_model2.get(f"resblock_{i}_ln_2")
    if act1 is not None and act2 is not None:
        diff = torch.norm(act1 - act2.to(dtype=torch.float32))
        print(f"LayerNorm2 difference: {diff.item()}")
    else:
        print("Missing ln2 activation")
    
    # Compare MLP first linear layer output.
    act1 = activations_model1.get(f"encoder_layer_{i}_mlp_fc1")
    act2 = activations_model2.get(f"resblock_{i}_mlp_fc1")
    if act1 is not None and act2 is not None:
        diff = torch.norm(act1 - act2.to(dtype=torch.float32))
        print(f"MLP FC1 difference: {diff.item()}")
    else:
        print("Missing MLP fc1 activation")
    
    # Compare MLP activation (GELU/QuickGELU).
    act1 = activations_model1.get(f"encoder_layer_{i}_mlp_act")
    act2 = activations_model2.get(f"resblock_{i}_mlp_gelu")
    if act1 is not None and act2 is not None:
        diff = torch.norm(act1 - act2.to(dtype=torch.float32))
        print(f"MLP Activation difference: {diff.item()}")
    else:
        print("Missing MLP activation (GELU)")
    
    # Compare MLP second linear layer output.
    act1 = activations_model1.get(f"encoder_layer_{i}_mlp_fc2")
    act2 = activations_model2.get(f"resblock_{i}_mlp_fc2")
    if act1 is not None and act2 is not None:
        diff = torch.norm(act1 - act2.to(dtype=torch.float32))
        print(f"MLP FC2 difference: {diff.item()}\n")
    else:
        print("Missing MLP fc2 activation\n")


""" # ----------------------------------------------------------------------------
# Now, compare parameters of the corresponding layers.
def compare_module_parameters(module1, module2, tol=1e-5):
    params1 = dict(module1.named_parameters())
    params2 = dict(module2.named_parameters())
    for name, p1 in params1.items():
        p2 = params2.get(name)
        if p2 is None:
            print(f"Parameter '{name}' not found in second module.")
        else:
            if torch.allclose(p1.detach().cpu(), p2.detach().cpu(), atol=tol):
                print(f"Parameter '{name}' matches.")
            else:
                print(f"Parameter '{name}' differs.")

print("=== Parameters Comparison ===")
# Compare high-level layers.
print("Comparing patch embedding parameters:")
compare_module_parameters(model1.vision_tower.vision_model.embeddings.patch_embedding, model2.conv1)

print("\nComparing pre-layer norm parameters:")
compare_module_parameters(model1.vision_tower.vision_model.pre_layrnorm, model2.ln_pre)

print("\nComparing post-layer norm parameters:")
compare_module_parameters(model1.vision_tower.vision_model.post_layernorm, model2.ln_post)

# Compare parameters for each internal block.
for i in range(24):
    print(f"\n--- Block {i} Parameters Comparison ---")
    
    # Compare first layer normalization parameters.
    print("LayerNorm1 parameters:")
    compare_module_parameters(encoder_layers_model1[i].layer_norm1, resblocks_model2[i].ln_1)
    
    # Compare attention output projection parameters.
    print("Attention Out Projection parameters:")
    compare_module_parameters(encoder_layers_model1[i].self_attn.out_proj, resblocks_model2[i].attn.out_proj)
    
    # Compare second layer normalization parameters.
    print("LayerNorm2 parameters:")
    compare_module_parameters(encoder_layers_model1[i].layer_norm2, resblocks_model2[i].ln_2)
    
    # Compare MLP first linear layer parameters.
    print("MLP FC1 parameters:")
    compare_module_parameters(encoder_layers_model1[i].mlp.fc1, resblocks_model2[i].mlp.c_fc)
    
    # Compare MLP second linear layer parameters.
    print("MLP FC2 parameters:")
    compare_module_parameters(encoder_layers_model1[i].mlp.fc2, resblocks_model2[i].mlp.c_proj)
 """
# ----------------------------------------------------------------------------
# Compare the final outputs of both models.
print("=== Final Output Comparison ===")
print(f"Model1 output: shape {output_model1.shape}, norm {torch.norm(output_model1)}")
print(f"Model2 output: shape {output_model2.shape}, norm {torch.norm(output_model2)}")
# Here, we compare model1 output to the transpose of model2 output; adjust if needed.
final_diff = torch.norm(output_model1 - output_model2.T.to(dtype=output_model1.dtype))
print(f"Final output difference: {final_diff.item()}")


direct
direct
direct
direct
direct
direct
direct
direct
direct
direct
direct
direct
direct
direct
direct
direct
direct
direct
direct
direct
direct
direct
direct
direct
CLIPVisionTower(
  (vision_tower): CLIPVisionModel(
    (vision_model): CLIPVisionTransformer(
      (embeddings): CLIPVisionEmbeddings(
        (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
        (position_embedding): Embedding(577, 1024)
      )
      (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (encoder): CLIPEncoder(
        (layers): ModuleList(
          (0-23): 24 x CLIPEncoderLayer(
            (self_attn): CLIPAttention(
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
      

RuntimeError: The size of tensor a (576) must match the size of tensor b (768) at non-singleton dimension 1

In [6]:
import torch
import torch.nn as nn

# Assume model1 and model2 are defined, loaded, and set to evaluation mode.
# For example:
#   model1 = ...  # a CLIP encoder-based model with: vision_tower.vision_model.encoder.layers (CLIPEncoderLayer)
#   model2 = ...  # a VisionTransformer model with: transformer.resblocks (ResidualAttentionBlock)

model1.eval()
model2.eval()

# Set the device.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ----------------------------------------------------------------------------
# Define a helper function to compare outputs of two modules on a fixed dummy input.
# For each module, we move it to CUDA, run the dummy input, then move it back to CPU.
def compare_modules(module1, module2, dummy_input, module_label=""):
    with torch.no_grad():
        # Evaluate module1 on CUDA and move the output back to CPU.
        module1.to(device)
        out1 = module1(dummy_input)[0]
        out1 = out1.cpu()
        module1.to("cpu")
        
        # Evaluate module2 on CUDA and move the output back to CPU.
        module2.to(device)
        out2 = module2(dummy_input)
        out2 = out2.cpu()
        module2.to("cpu")
        
    diff = torch.norm(out1 - out2.to(dtype=out1.dtype))
    print(f"{module_label} difference: {diff.item():.6f}")
    print(f"Module1 {module_label} output norm: {torch.norm(out1).item():.6f}")
    print(f"Module2 {module_label} output norm: {torch.norm(out2).item():.6f}\n")
    return out1, out2, diff

# ----------------------------------------------------------------------------

# For post-layernorm, assume a transformer input shape, e.g., (1, 197, 768); adjust if needed.
dummy_transformer_input = 100 * torch.randn(1, 577, 1024, device=device, dtype=torch.float16)

compare_modules(model1.vision_tower.vision_model.post_layernorm,
                model2.ln_post,
                dummy_transformer_input,
                "Post-LayerNorm")

# ----------------------------------------------------------------------------
# Internal Transformer/Encoder Block Comparisons
encoder_layers_model1 = model1.vision_tower.vision_model.encoder.layers
resblocks_model2 = model2.transformer.resblocks

# Create a dummy input for transformer blocks (adjust shape as required).
model1.to(device)
dummy_block_input = 100 * torch.randn(1, 577, 1024, device=device, dtype=torch.float16)
model1.to("cpu")

for i in range(len(encoder_layers_model1)):
    print(f"=== Comparing Block {i} as a whole ===")

    # Compare attention's output projection.
    compare_modules(encoder_layers_model1[i].self_attn,
                    resblocks_model2[i].attn,
                    dummy_block_input,
                    f"Block {i} Attn Out")
    print(f"--- Block {i} internal modules ---")
    # Compare first layer normalization.
    compare_modules(encoder_layers_model1[i].layer_norm1,
                    resblocks_model2[i].ln_1,
                    dummy_block_input,
                    f"Block {i} LN1")
    
    # Compare attention's output projection.
    compare_modules(encoder_layers_model1[i].self_attn.out_proj,
                    resblocks_model2[i].attn.out_proj,
                    dummy_block_input,
                    f"Block {i} Attn Out")
    
    # Compare second layer normalization.
    compare_modules(encoder_layers_model1[i].layer_norm2,
                    resblocks_model2[i].ln_2,
                    dummy_block_input,
                    f"Block {i} LN2")
    
    # Compare MLP first linear layer.
    compare_modules(encoder_layers_model1[i].mlp.fc1,
                    resblocks_model2[i].mlp.c_fc,
                    dummy_block_input,
                    f"Block {i} MLP FC1")
    
    # Compare MLP activation function.
    compare_modules(encoder_layers_model1[i].mlp.activation_fn,
                    resblocks_model2[i].mlp.gelu,
                    dummy_block_input,
                    f"Block {i} MLP Activation")
    
    # Compare MLP second linear layer.
    compare_modules(encoder_layers_model1[i].mlp.fc2,
                    resblocks_model2[i].mlp.c_proj,
                    dummy_block_input,
                    f"Block {i} MLP FC2")

# ----------------------------------------------------------------------------
# Final Output Comparison Using a Fixed Dummy Image.
with torch.no_grad():
    # Evaluate model1 on CUDA then move it back to CPU.
    model1.to(device)
    final_out1 = model1(dummy_image)
    final_out1 = final_out1.cpu()
    model1.to("cpu")
    
    # Evaluate model2 on CUDA then move it back to CPU.
    model2.to(device)
    final_out2 = model2(dummy_image)
    final_out2 = final_out2.cpu()
    model2.to("cpu")

print("=== Final Output Comparison ===")
print(f"Model1 final output: shape {final_out1.shape}, norm {torch.norm(final_out1).item():.6f}")
print(f"Model2 final output: shape {final_out2.shape}, norm {torch.norm(final_out2).item():.6f}")
final_diff = torch.norm(final_out1 - final_out2.T.to(dtype=final_out1.dtype))
print(f"Final output difference: {final_diff.item():.6f}")


RuntimeError: expected scalar type Half but found Float

## Here play around with LLava

In [5]:
# Layer where to extract infos on patches
select_layer = -2
# Params
prompt = "Describe the image focusing on main subjects. ignore background"
image_file = "images/catdog.png"
max_new_tokens = 512
num_beams = 1 # numer of path of decision, less faster
sep =  ","
temperature = 0 # 0 lowest, det
top_p = None
images_embeds = True # If provided image embeds

image_token_se = DEFAULT_IM_START_TOKEN + DEFAULT_IMAGE_TOKEN + DEFAULT_IM_END_TOKEN
# Making prompt in correct format
if IMAGE_PLACEHOLDER in prompt:
    if model.config.mm_use_im_start_end:
        prompt = re.sub(IMAGE_PLACEHOLDER, image_token_se, prompt)
    else:
        prompt = re.sub(IMAGE_PLACEHOLDER, DEFAULT_IMAGE_TOKEN, prompt)
else:
    if model.config.mm_use_im_start_end:
        prompt = image_token_se + "\n" + prompt
    else:
        prompt = DEFAULT_IMAGE_TOKEN + "\n" + prompt

# Derive necessary conv
if "llama-2" in model_name.lower():
    conv_mode = "llava_llama_2"
elif "mistral" in model_name.lower():
    conv_mode = "mistral_instruct"
elif "v1.6-34b" in model_name.lower():
    conv_mode = "chatml_direct"
elif "v1" in model_name.lower():
    conv_mode = "llava_v1"
elif "mpt" in model_name.lower():
    conv_mode = "mpt"
else:
    conv_mode = "llava_v0"

if conv_mode is not None and conv_mode != conv_mode:
    print(
        "[WARNING] the auto inferred conversation mode is {}, while `--conv-mode` is {}, using {}".format(
            conv_mode, conv_mode, conv_mode
        )
    )
else:
    conv_mode = conv_mode

# Load conversation mode standard template 
conv = conv_templates[conv_mode].copy()
conv.append_message(conv.roles[0], prompt)
conv.append_message(conv.roles[1], None)
prompt = conv.get_prompt()
print(prompt)

# Load images from online or local
image_files = image_parser(image_file, sep)
images = load_images(image_files)
image_sizes = [x.size for x in images]
# Convert images to format b, 3, h, w (h = w) with resizing or padding
images_tensor = process_images(
    images,
    image_processor,
    model.config
).to(model.device, dtype=torch.float16)


model_CLIP.eval()
vision_enc.eval()
# Tokenize prompt
input_ids = (
    tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors="pt")
    .unsqueeze(0)
    .to(device)
)


# Use CLIP Model
if images_embeds:
    model.to("cuda")
    output_pt = vision_enc(images_tensor.to(device))
    model.to("cpu")
    torch.cuda.empty_cache()
    #spatial_features = torch.randn((1, 576, 1024))# if want to manually edit features, we want b, nr_spat (576), d (1024)
    # Need to handle some memory movement :/
    gc.collect()
    torch.cuda.empty_cache()

   # prs.reinit()  # Reinitialize any hooks if required
    with torch.no_grad():
        model_CLIP.to("cuda")
        spatial_features = model_CLIP.encode_image(
                images_tensor.to(device), 
                attn_method='head',
                normalize=False
            )
        print(spatial_features.shape)
        model_CLIP.to("cpu")

        attentions, mlps = prs.finalize(spatial_features)  # attentions: [1, 12, 197, 16, 512], [b, l, n, h, d], mlps: [1, 13, 512], [b, l + 1, d]
        attentions = einops.rearrange(attentions, "b l n h d -> b l h n d")
        # Compute intermediate features from select_layer
        print(attentions.shape)
        print(mlps.shape)
        for k in range(1, 24 - 2):
            if k == 1:
                spatial_features = attentions[:, k, :, 1:, :].sum(1) + mlps[:, k, 1:, :]
            else:
                spatial_features += spatial_features + attentions[:, k, :, 1:, :].sum(1) + mlps[:, k, 1:, :]

            print(spatial_features.norm())
        #spatial_features = (attentions[:, :-2, :, 1:, :].sum(1).sum(1) + mlps[:, :-1, 1:, :].sum(1))
        print(spatial_features.shape)
        print(spatial_features.norm())
        print("MSE Loss:", mse_loss(output_pt.to("cpu"), spatial_features.to("cpu")).item())

        print(spatial_features.norm())
        print(attentions.shape, mlps.shape)
        
    # Move model back to gpu
    images_tensor = ct2 = output_pt#activations_model2.get(f"resblock_{23}")[:, 1:, :].to(device)
    print(images_tensor.shape)


# Generate an answer by using full model LLava
model.to("cuda")
with torch.inference_mode():
    output_ids = model.generate(
        input_ids,
        images=images_tensor,
        image_sizes=image_sizes,
        do_sample=True if temperature > 0 else False,
        temperature=temperature,
        top_p=top_p,
        num_beams=num_beams,
        max_new_tokens=max_new_tokens,
        use_cache=True,
        images_embeds = images_embeds # If want to give images embeds already precomputed TODO: Only support 1 image

    )

# Print the output
outputs = tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0].strip()
print(outputs)

A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the human's questions. USER: <image>
Describe the image focusing on main subjects. ignore background ASSISTANT:
dict_keys([])
dict_keys([])
head


NameError: name 'peinr' is not defined